In [73]:
from bs4 import BeautifulSoup as bs
import lxml
from random import choice
import random
import numpy as np
import time
import glob
import matplotlib.pyplot as plt

In [74]:
def ReadFile(data_set_number):
    
    if data_set_number==0:
        file_path = 'a280-n1395.txt'
    elif data_set_number==1:    
        file_path = 'a280-n279.txt'
    elif data_set_number==2: 
        file_path = 'a280-n2790.txt'
    elif data_set_number==3: 
        file_path = 'fnl4461-n22300.txt'
    elif data_set_number==4: 
        file_path = 'fnl4461-n4460.txt'
    elif data_set_number==5:  
        file_path = 'fnl4461-n44600.txt'
    elif data_set_number==6: 
        file_path = 'pla33810-n169045.txt'
    elif data_set_number==7: 
        file_path = 'pla33810-n33809.txt'
    elif data_set_number==8: 
         file_path = 'pla33810-n338090.txt'

    with open(file_path, 'r') as file:

        file_content = file.readlines()

    problem_info = {}
    node_coord_section = []
    items_section=[]

    for line in file_content:
        line = line.strip()
        if line.startswith('PROBLEM NAME:'):
            problem_info['PROBLEM_NAME'] = line.split(':')[-1].strip()
            
        elif line.startswith('KNAPSACK DATA TYPE:'):
            problem_info['KNAPSACK_DATA_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('DIMENSION:'):
            problem_info['DIMENSION'] = int(line.split(':')[-1].strip())
           
        elif line.startswith('NUMBER OF ITEMS:'):
            problem_info['NUMBER_OF_ITEMS'] = int(line.split(':')[-1].strip())
            
        elif line.startswith('CAPACITY OF KNAPSACK:'):
            problem_info['CAPACITY_OF_KNAPSACK'] = int(line.split(':')[-1].strip())

        elif line.startswith('MIN SPEED:'):
            problem_info['MIN_SPEED'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('MAX SPEED:'):
            problem_info['MAX_SPEED'] = float(line.split(':')[-1].strip())
          
        elif line.startswith('RENTING RATIO:'):
            problem_info['RENTING_RATIO'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('EDGE_WEIGHT_TYPE:'):
            problem_info['EDGE_WEIGHT_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('NODE_COORD_SECTION'):
            break

    for line in file_content[len(problem_info) + 1:]:

        if line.startswith('ITEMS SECTION'):
            break

        if line.strip():
            index, x, y = map(int, line.split())
            node_coord_section.append((index, x, y))


    for line in file_content[len(node_coord_section)+len(problem_info) +2 :]:
        if line.strip():
            index, x, y, z = map(int, line.split())
            items_section.append((index, x, y,z))
    
#     print(problem_info)#--------test
#     print(node_coord_section)#--------test
#     print(items_section)#--------test
    
    dimension = problem_info['DIMENSION']
    number_of_items= problem_info['NUMBER_OF_ITEMS']
    capacity_of_knapsack = problem_info['CAPACITY_OF_KNAPSACK']
    min_speed = problem_info['MIN_SPEED']
    max_speed = problem_info['MAX_SPEED']
    renting_ration=problem_info['RENTING_RATIO']
    
    return  dimension,\
            number_of_items,\
            capacity_of_knapsack,\
            min_speed,max_speed,\
            renting_ration,\
            node_coord_section,\
            items_section

In [75]:
dimension, number_of_items, capacity_of_knapsack, min_speed,max_speed, renting_ratio, node_coord_section, items_section = ReadFile(0)

In [76]:
print(node_coord_section[0])
print(type(node_coord_section[0]))
print(len(node_coord_section))
print(node_coord_section[0][2])
print(dimension)

(1, 288, 149)
<class 'tuple'>
280
149
280


<h3>Construct Distance Matrix</h3>

In [77]:
import math

In [78]:
D = []
for i, source in enumerate(node_coord_section):
    source_x = source[1]
    source_y = source[2]
    row = []
    j = 0
    while j < len(node_coord_section):
        if i==j:
            row.append(0)
        else:
            dest_x = node_coord_section[j][1]
            dest_y = node_coord_section[j][2]
            dist = math.sqrt((dest_x - source_x)**2 + (dest_y - source_y)**2)
            row.append(int(dist))
        j += 1
    D.append(row)

In [62]:
print(D)

[[0, 20, 24, 32, 32, 42, 55, 63, 61, 70, 78, 86, 94, 101, 92, 100, 116, 124, 132, 140, 148, 141, 125, 117, 133, 149, 157, 165, 172, 184, 184, 185, 198, 208, 224, 224, 232, 232, 232, 232, 232, 232, 233, 249, 248, 248, 248, 248, 248, 248, 256, 256, 256, 256, 256, 256, 257, 258, 250, 234, 236, 245, 252, 261, 262, 270, 276, 274, 282, 284, 286, 288, 290, 292, 294, 287, 297, 300, 283, 277, 274, 272, 269, 267, 264, 256, 259, 262, 266, 266, 269, 272, 284, 291, 298, 302, 295, 288, 280, 270, 266, 263, 259, 255, 248, 241, 238, 252, 259, 249, 246, 254, 251, 244, 241, 248, 239, 237, 191, 189, 187, 186, 185, 184, 184, 172, 164, 156, 156, 148, 140, 132, 124, 118, 120, 122, 125, 128, 132, 125, 129, 134, 139, 144, 150, 145, 140, 136, 141, 153, 176, 173, 168, 165, 167, 170, 193, 196, 199, 202, 209, 213, 217, 221, 226, 240, 250, 257, 257, 249, 242, 242, 234, 223, 205, 182, 179, 170, 170, 179, 186, 190, 194, 198, 203, 201, 208, 218, 212, 206, 197, 188, 181, 172, 167, 161, 166, 161, 155, 149, 155, 147, 135

<h3>Item Matrix</h3>
Help locate which items are in which city

In [79]:
print(len(items_section))

1395


In [109]:
#Get values and weights of items
value = []
weight = []
for item in items_section:
    value.append(item[1])
    weight.append(item[2])
print(len(value))
print(len(weight))

1395
1395


In [110]:
item_location = []
# for i, item in enumerate(items_section):
#     l = [j[3]-1 for j in item if ]
i = 0
while i < len(node_coord_section):
    row = [item[0]-1 for j,item in enumerate(items_section) if item[3]-1 == i]
    item_location.append(row)
    i += 1


In [125]:
# print(len(item_location))
# print(item_location)

<h3>Generate Random Solution for tsp or kp based on the flag</h3>

In [28]:
def GenerateRandomSolution(n, isTSP):
    solution = []
    if isTSP:
        while len(solution) < n:
            g = np.random.randint(n)
            #As TSP is permutation problem only one instance of a city is allowed the below check is implemented
            if g not in solution:
                solution.append(g)
    else:
        while len(solution) < n:
            g = np.random.randint(0,2)
            #As packing plan is 0 for item not picked up and 1 for item picked up create list of 0's and 1's of size n
            solution.append(g)
    return solution

In [82]:
tsp_population = GenerateRandomSolution(dimension, True)
kp_population = GenerateRandomSolution(number_of_items, False)

<h3>Generate a population of "size" number of random solutions </h3>

In [37]:
def GeneratePopulation(size, num_cities, num_items):
    tsp_pop = []
    kp_pop = []
    for i in range(size):
        tsp_pop.append(GenerateRandomSolution(num_cities,True))
        kp_pop.append(GenerateRandomSolution(num_items,False))
    return tsp_pop,kp_pop

In [83]:
tsp_population, kp_population = GeneratePopulation(500,dimension,number_of_items)

<h3>Calculate total profit of a packing plan</h3>

In [126]:
def eval_profit(packing_plan, value_list):
    picked_values = np.multiply(packing_plan,value_list)
#     print(picked_values)
    profit = sum(picked_values)
    return profit

In [127]:
eval_profit(kp_population[0],value)

336501

<h3>Calculate the speed based on the weight</h3>

In [85]:
def eval_speed(w):
    if w <= capacity_of_knapsack:
        Speed = (max_speed - (w*(max_speed-min_speed)/capacity_of_knapsack))
    else:
        Speed = min_speed
    return Speed

In [50]:
eval_speed(100000)

0.858714933831494

<h3>Calculate the time for journey</h3>
Affected by the items picked up

In [128]:
def eval_time(tour, packing_plan, item_location, item_weight):
    total_weight = 0
    total_time = 0
    for i,city in enumerate(tour):
        current_weight = []
        if i < dimension-1:
            
            items = item_location[i]
#             print(items)
            if items:
#                 for j in items:
#                     print(packing_plan[j])
#                     print(item_weight[j])
#                     current_weight.append(packing_plan[j]*item_weight[j])
                current_weight = [packing_plan[j]*item_weight[j] for j in items]
                total_weight += sum(list(current_weight))
            
            speed = eval_speed(total_weight)
            total_time += D[i][i+1]/speed
        else:
#             current_weight = 0
            items = item_location[i]
            if items:
                current_weight = [packing_plan[j]*item_weight[j] for j in items]
                total_weight += sum(list(current_weight))
            
            speed = eval_speed(total_weight)
            total_time += D[i][0]/speed
    return total_time

In [64]:
eval_time(tsp_population[0], kp_population[0], item_location, weight)

9387.768041331798

In [87]:
def eval_fitness(profit, time):
    fitness = profit - (renting_ratio*time)
    return fitness

In [67]:
eval_fitness(352813,9387.768041331798)

-329677.73660482175

<h6>Evaluate Fitness of entire population</h6>

In [94]:
def eval_population_fitness(tsp_pop, kp_pop):
    pop_fitness = []
    for tour,packing_plan in zip(tsp_pop,kp_pop):
        time = eval_time(tour, packing_plan, item_location, weight)
        profit = eval_profit(packing_plan, value)
        fitness = eval_fitness(profit, time)
        pop_fitness.append(fitness)
    return pop_fitness

<h6>Crossover functions</h6>
Crossover with fix for tsp
Normal crossover for kp

In [89]:
def CrossoverWithFix(a,b):
    #Choose a random point for crossover
    p = np.random.randint(1,len(a))
    #Split the chromosomes into 2 sections based on the crossover point
    a_left = a[:p]
    a_right = a[p:]
    b_left = b[:p]
    b_right = b[p:]
    #Form the children by combining the left section of one parent with the right section of the other parent
    a_new = a_left + b_right
    b_new = b_left + a_right
    #find missing and conflicting elements of a_new
    a_missing = [i for i in a if i not in a_new]
    a_conflict = [i for i in a_left if i in b_right]
    #fix a_new by replacing the conflicting elements with the missing elements
    for element in a_new:
        if element in a_conflict:
            copy = element
            a_new[a_new.index(element)] = choice([i for i in a_missing if i not in a_new])
            a_conflict.remove(copy)

    #find missing and conflicting elements of b_new
    b_missing = [i for i in b if i not in b_new]
    b_conflict = [i for i in b_left if i in a_right]
    #fix b_new by replacing the conflicting elements with the missing elements
    for element in b_new:
        if element in b_conflict:
            copy = element
            b_new[b_new.index(element)] = choice([i for i in b_missing if i not in b_new])
            b_conflict.remove(copy)
    
    return a_new,b_new

In [90]:
def Crossover(a,b):
    #Choose a random point for crossover
    p = np.random.randint(1,len(a))
    #Split the chromosomes into 2 sections based on the crossover point
    a_left = a[:p]
    a_right = a[p:]
    b_left = b[:p]
    b_right = b[p:]
    #Form the children by combining the left section of one parent with the right section of the other parent
    a_new = a_left + b_right
    b_new = b_left + a_right
    
    return a_new,b_new

<h6>Mutation Function</h6>

In [138]:
def SwapMutation(chromosome):
    #Choose 2 indices p1 and p2 randomly and p1 != p2
    p1 = np.random.randint(len(chromosome))
    p1_list = []
    p1_list.append(p1)
    p2 = choice([i for i in range(len(chromosome)) if i not in p1_list])
    #Swap the values at the p1 and p2 indices
    temp = chromosome[p1]
    chromosome[p1] = chromosome[p2]
    chromosome[p2] = temp
    return chromosome

<h6>Selection Function</h6>

In [96]:
def RankBasedSelection(tsp_pop,kp_pop,pop_fitness):
    #Combine the population and its fitness
    fitness_population = sorted(zip(pop_fitness, tsp_pop, kp_pop))
    #Sort the combined population and fitness based on the fitness values
    sorted_tsp_pop = [y for x,y,z in fitness_population]
    sorted_kp_pop = [z for x,y,z in fitness_population]
    sorted_pop_fitness = [x for x,y,z in fitness_population]
    rank = []
    #Assign ranks to the solutions in the population
    for i in range(len(fitness_population)):
        rank.append(i+1)
    rank_total = sum(rank)
    probability = []
    #Find the probabilities of choosing a solution from the population based on its rank
    for r in rank:
        probability.append(r/rank_total)
    indeces = np.arange(len(fitness_population))
    #Choose 2 parents randomly based on the rank based probabilities calculated
    choice_1, choice_2 = np.random.choice(indeces,2,replace=True,p=probability)
#     parent_1, parent_2 = sorted_population[choice_1], sorted_population[choice_2]
#     return parent_1,parent_2
    return choice_1,choice_2

<h3>Replacement Funtion</h3>

In [92]:
def ReplaceWeakest(tsp_pop, kp_pop, pop_fitness, child_tsp, child_kp, child_fitness):
    #Find the worst fitness in the population
    worst_fitness = min(pop_fitness)
    #Replace the worst solution if the new candidate solution is better or the same
    if child_fitness >= worst_fitness:
        list_index = pop_fitness.index(worst_fitness)
        del pop_fitness[list_index]
        del tsp_pop[list_index]
        del kp_pop[list_index]
        tsp_pop.append(child_tsp)
        kp_pop.append(child_kp)
        pop_fitness.append(child_fitness)
    return tsp_pop, kp_pop, pop_fitness

<h3>10000 runs of GA - Rank selection, Crossover, Replace Weakest</h3>

In [139]:
tsp_population, kp_population = GeneratePopulation(500,dimension,number_of_items)
print(type(kp_population))
population_fitness = eval_population_fitness(tsp_population, kp_population)
avg_fitness = sum(population_fitness)/dimension
print("Average Fitness - ",avg_fitness)
for i in range(10000):
    p1, p2 = RankBasedSelection(tsp_population,kp_population,population_fitness)
    tsp_p1 = tsp_population[p1]
    tsp_p2 = tsp_population[p2]
#     print("TSP P1 - ",tsp_p1)
#     print("TSP P2 - ",tsp_p2)
    kp_p1 = kp_population[p1]
    kp_p2 = kp_population[p2]
#     print("KP P1 - ",kp_p1)
#     print("KP P2 - ",kp_p2)
    tsp_child1, tsp_child2 = CrossoverWithFix(tsp_p1,tsp_p2)
    kp_child1, kp_child2 = Crossover(kp_p1,kp_p2)
#     print("TSP child - ",tsp_child)
#     print(len(tsp_child1))
#     print(type(tsp_child1))
#     print("KP child - ",kp_child)
#     print(len(kp_child1))
#     print(type(kp_child1))
    tsp_child1 = SwapMutation(tsp_child1)
    tsp_child2 = SwapMutation(tsp_child2)
    kp_child1 = SwapMutation(kp_child1)
    kp_child2 = SwapMutation(kp_child2)
    child1_time = eval_time(tsp_child1, kp_child1, item_location, weight)
    child1_profit = eval_profit(kp_child1, value)
    child1_fitness = eval_fitness(child1_profit, child1_time)
    tsp_population, kp_population, population_fitness = ReplaceWeakest(tsp_population, kp_population, population_fitness, tsp_child1, kp_child1, child1_fitness)
    
    child2_time = eval_time(tsp_child2, kp_child2, item_location, weight)
    child2_profit = eval_profit(kp_child2, value)
    child2_fitness = eval_fitness(child2_profit, child2_time)
    tsp_population, kp_population, population_fitness = ReplaceWeakest(tsp_population, kp_population, population_fitness, tsp_child2, kp_child2, child2_fitness)
#     ReplaceWeakest(tsp_pop, kp_pop, pop_fitness, child_tsp, child_kp, child_fitness)
    avg_fitness = sum(population_fitness)/dimension
    print(avg_fitness)

<class 'list'>
Average Fitness -  -553323.9818922085
-552936.9946426253
-552514.5246255934
-552184.6354201974
-551806.184232135
-551187.8421808053
-550998.4393153632
-550453.0940216085
-550245.8212430223
-549768.7521214903
-549093.8241124493
-548907.6073040898
-548490.4405750241
-548167.9205817956
-547945.2067793041
-547484.7922353055
-547184.1433088412
-546805.6454623078
-546574.7505840355
-546393.7801921816
-546116.9212362469
-545706.6967745365
-545263.4134777966
-544727.0248349807
-544511.2108165156
-544233.9391944788
-544053.8351857474
-543878.9135544845
-543733.6439061465
-543406.4001964309
-543164.8578590017
-542474.2001299219
-541982.1276533913
-541450.8304695509
-541205.8351650705
-540864.6842654884
-540482.3717695898
-540148.0742243445
-539596.0632713729
-539469.1459935637
-539061.2709439457
-538652.8567571606
-538336.6546170063
-538053.4303522962
-537939.9224287146
-537552.9995337614
-537108.5105609612
-536939.5643256876
-536705.5396718362
-536435.2125140503
-535867.014940523

-460295.93062945246
-460090.90604859695
-459966.9778756602
-459775.46968888765
-459623.24734090193
-459501.34773665713
-459136.33673209616
-458991.51321549696
-458849.62781350646
-458696.8513155992
-458418.1338034968
-458243.48898079974
-458154.951860482
-458005.29554044217
-457844.1435464506
-457621.46737361397
-457372.4014670009
-457139.1182580119
-457000.5706278728
-456890.8792933088
-456643.6541078906
-456272.58323096426
-456188.46708296967
-456064.76803536253
-455822.69477995473
-455735.68842971657
-455620.32388375973
-455566.7738596343
-455427.1473817518
-455114.9012016632
-455050.2392144023
-454950.0092527248
-454660.22226581746
-454552.6132665986
-454368.34526365733
-454188.34135375574
-454010.0989568342
-453910.71385409025
-453825.1783660878
-453695.0584938499
-453619.63311995286
-453489.5413526613
-453292.9211802578
-453117.9927723627
-452944.4310502615
-452729.3490950711
-452540.1005436197
-452402.79856462264
-452269.4664285244
-452168.2027516294
-452024.51059026195
-451774.

-377462.59041368694
-377281.05520982365
-377110.89564946695
-377009.5249036341
-376793.2832944466
-376671.69057609816
-376587.3197026083
-376519.1694385465
-376446.91416677996
-376402.57922081253
-376340.10889076785
-376199.8346975793
-376021.075520483
-375924.0511489593
-375754.17799079197
-375542.93593558454
-375327.328399058
-375214.23336186423
-375107.76535257103
-374835.4621535837
-374734.62105428526
-374622.54914809245
-374411.0672914754
-374077.79430771124
-373906.979870888
-373749.4738255705
-373691.44078236696
-373597.2835397164
-373273.3830027976
-373185.9536546815
-373048.10629105824
-372856.8542647768
-372775.67686423194
-372658.1565768874
-372522.88717903383
-372352.0495451293
-372276.0017528946
-372228.2426577243
-372153.39158576966
-372089.0093760065
-371977.28331009595
-371709.2540163172
-371483.36361913226
-371233.05215123505
-371155.18770433246
-371066.0591115099
-370936.7350019995
-370897.0064586338
-370751.6223164512
-370610.3363726281
-370397.9303751355
-370200.471

-317731.1883790899
-317537.08379122015
-317433.8977364264
-317334.12203620444
-317105.0100281749
-317021.7594023756
-316908.2854965457
-316787.55221054796
-316585.5548377561
-316458.67641177855
-316294.8804603651
-316199.2821692717
-316126.6503656497
-315967.4362914876
-315903.3019556822
-315686.8065065604
-315589.7026698406
-315470.21024234087
-315447.7090233338
-315398.9283823126
-315242.65902252484
-315137.310920845
-315007.9716537299
-314896.0783533013
-314797.26510206383
-314655.12071826286
-314599.8821246563
-314503.2726280507
-314296.823165964
-314209.87536030816
-314133.5435009817
-313966.01611966104
-313949.8669038865
-313825.40968727454
-313737.181632996
-313592.15055710234
-313522.0933155449
-313385.66419558274
-313260.1036673978
-313058.0030270757
-312920.7990818834
-312785.82145091175
-312618.4204979197
-312498.5930268926
-312345.0791222589
-312201.2447204126
-311968.30620698247
-311858.100826086
-311727.3915991285
-311677.3453254298
-311617.15056094783
-311513.6170602878


-267065.54941047274
-266935.7508392041
-266827.75474241073
-266781.2251144041
-266639.370942995
-266517.3754704138
-266474.0303052543
-266355.45393761253
-266240.0276600849
-266180.90779627865
-266078.1114564801
-266014.02711235045
-265918.57526093867
-265825.0888076582
-265759.45687880483
-265696.3097239655
-265610.98337695637
-265487.76851832046
-265436.82754917315
-265343.23650844186
-265308.20487052354
-265244.19300802605
-265203.4926573213
-265100.0698503652
-265002.90688757994
-264936.3875577475
-264752.7083591794
-264678.88201012334
-264555.57154713233
-264412.8471160401
-264338.7759669695
-264299.21902104805
-264193.54074990656
-264066.1685367968
-263954.32949676906
-263882.9397061071
-263811.9251785281
-263719.73582076083
-263684.8221545406
-263624.6447152346
-263532.5300854436
-263455.79736084474
-263393.3953667601
-263330.07802162005
-263281.4719767257
-263238.72726602823
-263177.62293896603
-263065.0251544531
-262980.90555378393
-262920.78380180564
-262833.7395964025
-26276

...

-88505.86821267092
-88502.39828510005
-88491.6091814014
-88471.66697575581
-88450.78419284344
-88444.57309134926
-88427.06251760504
-88408.23454669803
-88386.62623424936
-88380.49913182523
-88368.21696738315
-88349.93912723067
-88346.0640692378
-88334.73430235816
-88322.16778209354
-88305.32495255923
-88294.33707140193
-88278.77397698443
-88268.24577190565
-88259.08259540038
-88240.30092089306
-88234.62584709236
-88223.34706470935
-88209.39623795844
-88202.091551507
-88185.73127711483
-88164.107043068
-88151.90842345118
-88146.01560123195
-88139.43902346726
-88135.80189529576
-88125.6402951369
-88119.63037485689
-88107.40484502359
-88091.4036315516
-88075.153793509
-88064.87083625408
-88050.70218710507
-88035.45192212852
-88026.18514998979
-88015.78671999925
-88014.15488002056
-87993.2414535244
-87977.64155939878
-87961.8504621692
-87947.5833440928
-87928.46303614034
-87922.0118767386
-87908.85785780687
-87893.1726445803
-87887.9335294158
-87875.0438458277
-87848.5934248363
-87836.7342

-83377.19510000342
-83372.39178477964
-83359.7462578711
-83353.8278738015
-83324.13597127848
-83300.86480440582
-83294.88076018692
-83286.51336072254
-83270.49572518074
-83258.77206900773
-83251.88261168101
-83243.36019732284
-83231.04707985026
-83223.22494496864
-83209.71372547129
-83200.33927703589
-83190.53085536874
-83186.4381348375
-83170.22300828312
-83155.3264136435
-83151.53064173377
-83149.2304400931
-83130.15935409577
-83125.47401737666
-83115.7589191767
-83106.97467797929
-83096.68833478006
-83082.14805920841
-83074.42991321263
-83072.2289569033
-83060.54950975378
-83052.37236589371
-83034.62150807808
-83023.69299111851
-83002.76168266164
-82983.60136493547
-82977.82641180881
-82965.91339658079
-82964.46977300267
-82951.50332097115
-82934.70613074605
-82933.27861472881
-82918.4068937833
-82916.50311613739
-82906.17521554037
-82903.84251933506
-82894.69449747722
-82880.01239767538
-82879.12893255829
-82874.5068370069
-82856.91235652538
-82850.24983856085
-82841.70461985761
-8

-78009.5745604063
-77989.17330155117
-77980.53736789855
-77965.85125545206
-77960.76786330157
-77946.76636767157
-77921.45974901074
-77904.88954291416
-77899.05403165714
-77875.33170010998
-77860.31663410093
-77839.00801939445
-77829.16885650076
-77824.0167390293
-77812.58951193646
-77803.28731296328
-77795.16419501217
-77784.26118701475
-77775.80710160112
-77765.94303122301
-77759.31283729606
-77738.98439661891
-77722.4707947932
-77713.24339433096
-77699.96822468522
-77689.31894091603
-77675.78837866521
-77665.7862527725
-77646.11504376717
-77632.09852788282
-77625.2178298136
-77614.2027153246
-77605.18598698491
-77593.04781301373
-77568.91756194417
-77555.7647946752
-77549.0317919519
-77543.65184866825
-77527.18258417542
-77518.22180260083
-77504.61203388762
-77491.06299306655
-77474.27086984662
-77461.29791367093
-77452.95297991562
-77431.10698667563
-77413.42513647677
-77396.04624573818
-77389.30416809928
-77374.49648694844
-77366.83727540042
-77354.89533823493
-77338.30547210704
-

-73246.0494064782
-73244.24058271214
-73237.18162475555
-73221.45933415675
-73216.20205509716
-73210.92996090346
-73202.6499453116
-73190.2997101883
-73182.1950592092
-73174.6620737341
-73166.48140151014
-73155.6462036198
-73146.88366310958
-73133.46155253511
-73115.34830767546
-73109.51456176091
-73105.57205554517
-73100.16550021316
-73087.02433029428
-73074.16071731473
-73066.65407716426
-73051.35528615952
-73040.07797596075
-73028.49143751407
-73018.25050501863
-73008.38849947108
-73006.43954968739
-72991.75107420949
-72984.33512509316
-72969.4974350954
-72954.68387316822
-72938.75285161335
-72928.1134118829
-72921.5350102888
-72914.46966603011
-72909.01006110031
-72900.16076218979
-72894.1092076698
-72874.71668118364
-72862.23035947474
-72852.80152663025
-72847.72937600358
-72838.05679607306
-72830.87596198362
-72827.58137949748
-72819.00821701651
-72806.36634957735
-72779.0852661319
-72768.82447798777
-72763.21366570656
-72756.96158881726
-72747.03666371274
-72741.08512333572
-727

-68628.97948434907
-68624.62530419452
-68620.36428932943
-68609.10288310146
-68597.63300413762
-68582.97153206838
-68579.38794982598
-68566.61667649302
-68559.41884393967
-68544.21328378582
-68539.45914503856
-68528.78562579889
-68514.09148307984
-68508.07181254956
-68484.37751086577
-68469.24328996136
-68457.01997685347
-68454.62455149779
-68440.02665770965
-68438.35138454268
-68426.59691019884
-68408.74537486109
-68394.77868845194
-68380.4121431504
-68367.65741871233
-68357.45735196483
-68347.1534732484
-68324.7381337632
-68322.72522284552
-68318.58282232452
-68312.25418143756
-68300.18275675889
-68296.92935415354
-68279.07265939127
-68267.68786228226
-68261.79299773727
-68251.27066909251
-68245.29392518685
-68240.14466354197
-68230.47189005163
-68219.08372981042
-68213.90205218717
